## Data and Task Description:
### DataSet1:
- **Columns**: `id_audit`, `id_ftp`, `value`, `field`
  - `id_audit`: Alarm ID, data is collected each time an alarm is raised.
  - `value`: Temperature data collected by sensors, operational range from -40°C to 125°C.
  - `field`: Name of the temperature sensor.
  - `id_ftp`: Not used.
  
### DataSet2:
- **Columns**: `id_audit`, `branch_header`, `value`
  - `id_audit`: Same as above.
  - `branch_header`: Can be used as the ID for the branch, values range from "Branch A" to "Branch H".
  - `value`: Power configuration of the branch, in watts (W) and decibels-milliwatts (dBm), respectively.
  
### DataSet3:
- **Columns**: `id_audit`, `customer`, `serial`
  - `id_audit`: Same as above.
  - `customer`: Values are 1 and 2, representing two different customers.
  - `serial`: Serial number of the hardware device, can be used as the device ID.

### Task:
1. This is an unsupervised learning task; there is no labeled data to evaluate and validate the model. No performance metric.
2. Temperature data from sensors has been preprocessed with median filtering.

### Goals:
#### Prio 1: Anomaly detection
1. Support troubleshooters and design to understand more about this problem.
2. Suggested work:
- Do a cluster analysis
- Find outliers, which radios and sensors? (serial numbers in Dataset III)
- Identify why they are outliers
3. Required datasets: Dataset I, Dataset II, Dataset III

#### Prio 2:  Correlation with power class configurations
1. How does the sensor temperature data correlate with different power class configurations?
2. Is there a delta among similar configurations? 
3. Required datasets: Dataset I, Dataset II

#### Prio 3:  Predict sensor temp given the power class configuration
1. Can you use a supervised learning approach to predict the sensor temperature of a radio based on its power class configuration?
2. Required datasets: Dataset I, Dataset II


### Read data from CSV files

In [8]:
import pandas as pd
import sklearn

new_columns_name = ['id_field', 'id_ftp', 'id_audit', 'temperature', 'unit', 'sensor']
columns_to_read = ['id_audit', 'temperature', 'sensor']
df1 = pd.read_csv('dataset_1_radioTemperatures_20210303.csv',
                  sep=';',
                  header=0,
                  usecols = columns_to_read,
                  names=new_columns_name,
                  skip_blank_lines=True,
                  skipinitialspace=True)

print(df1.shape)
#discard the sensor data which is out of the working range
# df2 = df1[(df1['temperature'] < -40) | (df1['temperature'] > 150)]
# print(df2)
df1 = df1[(df1['temperature'] >= -40) & (df1['temperature'] <= 150)]
print(df1.shape)
# print(df1['temperature'])

#read the table 3
new_columns_name = ['id_audit', 'customer', 'device']
df3 = pd.read_csv('dataset_3_radioId_20210303.csv',
                  sep=';',
                  header=0,
                  names=new_columns_name,
                  skip_blank_lines=True,
                  skipinitialspace=True)

#merge table 1 and 3
df_merged = pd.merge(df1, df3, on='id_audit', how='left')
print(df_merged.describe)
null_counts = df_merged.isnull().sum()
print(null_counts)

(3238259, 3)
(3236927, 3)
<bound method NDFrame.describe of          id_audit  temperature        sensor  customer  device
0             147         23.5   TSensor_D4_         2   17087
1             147         23.8   TSensor_D5_         2   17087
2             147         26.3   TSensor_D6_         2   17087
3             147         23.5   TSensor_D8_         2   17087
4             147         23.0   TSensor_D9_         2   17087
...           ...          ...           ...       ...     ...
3236922   1970516         87.6  TSensor_T68_         1   23312
3236923   1970516         77.0  TSensor_T69_         1   23312
3236924   1970516         78.9  TSensor_T70_         1   23312
3236925   1970516         79.1  TSensor_T71_         1   23312
3236926   1970516         77.8  TSensor_T72_         1   23312

[3236927 rows x 5 columns]>
id_audit       0
temperature    0
sensor         0
customer       0
device         0
dtype: int64


In [15]:
value_counts = df_merged['id_audit'].value_counts()
pd.set_option('display.max_rows', 1000)
print(value_counts)
with open('output.txt', 'w') as f:    
    f.write(value_counts.to_string())  

id_audit
147        48
1422336    48
1423071    48
1420617    48
1419186    48
           ..
1285951    24
1349385    24
1285950    24
1782690    24
1285952    24
Name: count, Length: 67455, dtype: int64


In [7]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

# dataframe contains features ['id_audit', 'sensor', 'temperature', 'device']

# First, calculate the temperature statistical features for each sensor
sensor_stats = df_merged.groupby('sensor')['temperature'].agg(['mean', 'std', 'min', 'max']).reset_index()

# Encode sensor names using OneHotEncoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# Create a transformer that combines OneHotEncoder with standardization
preprocessor = make_column_transformer(
    (encoder, ['sensor']),
    (StandardScaler(), ['mean', 'std', 'min', 'max']),
    remainder='passthrough'
)

# Initialize the KMeans clustering algorithm, setting the number of clusters to 8.
kmeans = KMeans(n_clusters=8, random_state=0)

# Create a pipeline that includes preprocessing and clustering steps
pipeline = make_pipeline(preprocessor, kmeans)

# Fit the pipeline using data that includes sensor names and temperature statistical features
pipeline.fit(sensor_stats)



c:\Users\ecaijun\openvino_env\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\ecaijun\openvino_env\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['sensor']),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  ['mean', 'std', 'min',
                                                   'max'])])),
                ('kmeans', KMeans(random_state=0))])

In [6]:
pd.set_option('display.max_rows', 100)
# Add the clustering labels to the sensor_stats DataFrame
sensor_stats['cluster'] = pipeline.named_steps['kmeans'].labels_

#Output the cluster assigned to each sensor name.
print(sensor_stats[['sensor', 'cluster']])

          sensor  cluster
0    TSensor_C1_        3
1   TSensor_D10_        0
2   TSensor_D11_        6
3   TSensor_D12_        1
4   TSensor_D13_        1
5   TSensor_D14_        1
6   TSensor_D15_        1
7   TSensor_D16_        1
8   TSensor_D17_        1
9   TSensor_D18_        2
10  TSensor_D19_        2
11   TSensor_D2_        3
12   TSensor_D3_        6
13   TSensor_D4_        0
14   TSensor_D5_        0
15   TSensor_D6_        0
16   TSensor_D7_        6
17   TSensor_D8_        0
18   TSensor_D9_        0
19  TSensor_P23_        5
20  TSensor_P24_        4
21  TSensor_P25_        4
22  TSensor_P26_        4
23  TSensor_P27_        7
24  TSensor_P28_        2
25  TSensor_P29_        2
26  TSensor_P30_        0
27  TSensor_P31_        5
28  TSensor_P32_        4
29  TSensor_P33_        4
30  TSensor_P34_        5
31  TSensor_P35_        2
32  TSensor_P36_        2
33  TSensor_P37_        2
34  TSensor_P38_        2
35  TSensor_P39_        1
36  TSensor_P40_        1
37  TSensor_